# Controlled-Z Pulse Calibration

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

This tutorial introduces how to use Quanlse to simulate the calibration for the controlled-Z (CZ) gate in real experiments. The outline of this tutorial is as follows:

- Background
- Preparation
- Initialize the Two-qubit Simulator
- Controlled-Z Gate Pulse Calibration
    - Calibrate the Pulse for Single-qubit Gates
    - Calibrate the Conditional-phase
    - Calibrate the Dynamical-Phase
- Generating Bell State Using Calibrated Pulses

## Background

In the previous calibration tutorial, we have introduced the calibration and characterization methods for the single-qubit. In this tutorial, we will introduce the pulse calibration method for the controlled-Z (CZ) gate. In superconducting quantum computing, the CZ gate is a commonly used native two-qubit gate, which is easier to be implemented on the superconducting platform. The basic principle is to tune the eigenfrequency of qubits by adjusting the magnetic flux, so that the $|11\rangle $ state and the $|20\rangle$ ($|02\rangle$) state resonate and undergo avoided crossing, and eventually accumulate the phase of $\pi$ on the $|11\rangle$ state. The role of the CZ gate can be understood as that the $|1\rangle$ state phase of the target qubit increases by $\pi$ when the control qubit is at the $|1\rangle$ state. The corresponding matrix of the CZ gate in the two-qubit computational subspace is represented as:

$$
U_{\rm CZ} = |0\rangle\langle 0| \otimes I + |1\rangle\langle1| \otimes \hat{\sigma}^z = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{bmatrix}. 
$$

So far, Quanlse has provided the pulse optimization cloud service for the CZ gate, and introduced the relevant principles in the corresponding tutorial, which can be viewed in detail by users by clicking [Controlled-Z gate](https://quanlse.baidu.com/#/doc/tutorial-cz). In this tutorial, we will introduce the calibration method of CZ gates in real experiments.

## Preparation

After successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import the dependent packages
import numpy
from math import pi

# Import the two-qubit simulator
from Quanlse.Simulator.PulseSim2Q import pulseSim2Q

# Import the basis function to generate state bector
from Quanlse.Utils.Functions import basis

# Import the center-aligned pulse scheduling strategy
from Quanlse.Scheduler.Superconduct.PipelineCenterAligned import centerAligned

# Import the two qubit gate calibration functions
from Quanlse.Calibration.TwoQubit import czCaliCondPhase, czCaliDynamicalPhase, \
    czCaliCondPhaseJob, caliSingleQubitGates

# Import the operator for generating basis string list
from Quanlse.Utils.Functions import computationalBasisList, project

# Import the function for plot bar figures
from Quanlse.Utils.Plot import plotBarGraph

# Import the QOperation
from Quanlse.QOperation.FixedGate import H, CZ

## Initialize the Two-qubit Simulator

First, we need to initialize a two-qubit simulator. In Quanlse v2.1, we have already added a two-qubit simulator template. We can instantiate a `Quanlse.Simulator.PulseModel` object through the `Quanlse.Simulator.PulseSim2Q()` function, where the parameter `dt` represents the time step length for solving the Schrödinger equation, and the `frameMode` represents the reference frame used for simulation (`lab` and `rot` represent the lab frame and rotating frame respectively):

In [ ]:
# Sampling period.
dt = 0.01

# The indexes of qubits for calibration
q0 = 0
q1 = 1

# Instantiate the simulator object by a 3-qubit template.
model = pulseSim2Q(dt=dt, frameMode='lab')

The simulator models two directly coupled three-level qubits with predefined properties such as qubit frequencies, anharmonicity strengths, and the coupling strength between the qubits. These information are stored in the instantiated object `model`, and can be accessed by:

In [ ]:
print("Qubit frequency (GHz):\n    ", model.qubitFreq)
print("Microwave drive frequency (GHz):\n    ", model.driveFreq)
print("Qubit anharmonicity (GHz):\n    ", model.qubitAnharm)
print("Qubit coupling map (GHz):\n    ", model.couplingMap)

For ease of use, the calibrated control pulse parameters are also included in `model`, which can be accessed by the user through the `model.conf` property:

In [ ]:
print("Microwave control pulse parameters (a.u.):")
print(f"    q0: {model.conf['caliDataXY'][0]}")
print(f"    q1: {model.conf['caliDataXY'][0]}")
print("Flux control pulse parameters (a.u.):")
print(f"    q0: {model.conf['caliDataZ'][0]}")
print(f"    q1: {model.conf['caliDataZ'][0]}")
print("CZ gate control pulse parameters (a.u.):")
print(f"    q0q1: {model.conf['caliDataCZ'][(0, 1)]}")

The `pulseSim2Q()` method returns a pulse simulator type object `Quanlse.Simulator.PulseModel`, which inherits from class `Quanlse.Scheduler` ([click to view API](https://quanlse.baidu.com/Scheduler/Quanlse.Scheduler.html)), users can set the pulse scheduling strategy by themselves. Here we use the center-aligned strategy (`centerAligned`), and use the `addPipelineJob()` method in `model.pipeline` to add it to `model`, so that Quanlse Scheduler makes the pulses center-aligned:

In [ ]:
# Set the center-aligned scheduling sctrategy
model.pipeline.addPipelineJob(centerAligned)

Here, we set the property `model.savePulse` to `False` to turn off the pulse buffering for the quantum gates:

In [ ]:
# Prevent Quanlse Scheduler to cache the pulses
model.savePulse = False

## Pulse Calibration for Controlled-Z 

After initializing and configuring the simulation environment, we start the pulse calibration process for the CZ gate, which mainly includes the following steps:

  1. Calibrate the Pulse for Single-qubit Gates
  2. Calibrate the Conditional-phase
  3. Calibrate the Dynamical-Phase

### 1. Calibrate the Pulse for Single-qubit Gates

Since the superconducting qubits are not ideal two-level systems in real experiments, for weak anharmonicity qubits, energy leakage to the third energy level can take the state of the qubit out of the computational subspace, so we need to consider the error introduced by energy leakage. In the [DRAG pulse](https://quanlse.baidu.com/#/doc/tutorial-drag) chapter, we have introduced the principle and method of correcting the waveform of the driving pulse to eliminate the error of energy level leakage. In this tutorial, we will also use DRAG pulses to improve the fidelity of single-qubit gates.

In Quanlse v2.1, we provide a two-qubit calibration toolkit in the `Quanlse.Calibration.TwoQubit` module, in which we provide the function of DRAG pulse calibration in two-qubit systems, and users can use the `caliSingleQubitGates()` function to calibrate and obtain data of the calibrated pulses:

In [ ]:
q0ParaInit = [model.conf["caliDataXY"][q0]["piAmp"], model.conf["caliDataXY"][q0]["piAmp"]]
q1ParaInit = [model.conf["caliDataXY"][q1]["dragCoef"], model.conf["caliDataXY"][q1]["dragCoef"]]
bounds = [(0, 1), (-1, 1), (0, 1), (-1, 1)]

q0PiAmp, q0Drag, q1PiAmp, q1Drag, optGatesLoss = caliSingleQubitGates(
    model, q0, q1, bounds=bounds, q0ParaInit=q0ParaInit, q1ParaInit=q1ParaInit)

print(f"The optimal pi amp of q0 and q1 is {round(q0PiAmp, 6)} and {round(q1PiAmp, 6)}")
print(f"The optimal DRAG coefficient of q0 and q1 is {round(q0Drag, 6)} and {round(q1Drag, 6)}")
print(f"The minimal infidelity is {round(optGatesLoss, 6)}")

After completing the calibration of the $\pi$ and DRAG pulse parameters, we add the calibrated $\pi$ pulse amplitude and DRAG correction coefficient to `model.conf` through the following code:

In [ ]:
model.conf["caliDataXY"][q0]["piAmp"] = q0PiAmp
model.conf["caliDataXY"][q0]["dragCoef"] = q0Drag
model.conf["caliDataXY"][q1]["piAmp"] = q1PiAmp
model.conf["caliDataXY"][q1]["dragCoef"] = q1Drag

### 2. Calibrate the Conditional phase

In this section, we introduce how to calibrate the conditional phase, which is also the most important step to realize the CZ gate. We usually tune the frequency of each energy level through magnetic flux and change the phase of each quantum state $|ij\rangle$, the corresponding matrix form is as follows:

$$
{\rm CZ}_{\rm real} = \begin{bmatrix}
1 & 0 & 0 & 0 \\ 
0 & e^{i\theta_{01}} & 0 & 0 \\ 
0 & 0 & e^{i\theta_{10}} & 0 \\ 
0 & 0 & 0 & e^{i\theta_{11}}
\end{bmatrix}.
$$

Where $\theta_{ij}$ represents the phase obtained by the quantum state $|ij\rangle$. To implement the CZ gate, we first need to implement the conditional phase as $\pi$, that is to say, design the flux control trajectory and make $\theta_{11}=\pi$. The method is as follows: first, we apply an $X/2$ gate on the first qubit q0 to prepare it in the superposition state of $|0\rangle$ and $|1\rangle$; and simultaneously, apply an $X$ gate or $I$ gate on the second qubit q1 respectively; then, apply flux control to realize $ |11\rangle$ phase accumulation; finally execute an $X/2$ gate on q0 to change the coordinate representation and display the phase change, as shown in the following figure:

![fig:czCali_circuit](figures/cali-cz-circuit.png)

Where $\alpha_0$ and $\alpha_1$ are the magnetic flux tuning the first and second qubits, respectively. Through the measurement, we can obtain the final state when q0 is $|0\rangle$ or $|1\rangle$ state respectively:

$$
\left[R_x(\pi/2)\otimes I\right] \cdot |\psi\rangle_{\rm q1=|0\rangle} = \frac{1-e^{i\theta_{10}}}{2} |00\rangle - \frac{i(1+e^{i\theta_{10}})}{2} |10\rangle, \\
\left[R_x(\pi/2)\otimes I\right] \cdot |\psi\rangle_{\rm q1=|1\rangle} = \frac{e^{i\theta_{01}}-e^{i\theta_{11}}}{2} |01\rangle - \frac{i(e^{i\theta_{01}}+e^{i\theta_{11}})}{2} |11\rangle.
$$

It is obvious that when the $I$ gate is applied to q1, with $\theta_{10}=0$, the final state is $-i|10\rangle$, hence the measurement of q0 should give more counts of $|1 \rangle$. And when $X$ gate is applied on q1, if $\theta_{11}=\pi$ and $\theta_{01}=0$, $R_x(\pi/2) \cdot |\psi \rangle_{\rm q1=|1\rangle}=|01\rangle$, hence measuring q0 should give more counts of $|0\rangle$. Therefore, we can optimize the measurement result of q0 as a loss function to obtain the required conditional phase:

$$
{\rm Loss} = {\rm Prob_{q1=|0\rangle}(|01\rangle+|11\rangle)} + {\rm Prob_{q1=|1\rangle}(|00\rangle+|10\rangle)}.
$$

We also encapsulate the conditional phase calibration method `czCaliCondPhase()` in the `Quanlse.Calibration.TwoQubit` module. This method contains functions such as pulse scheduling, measurement and optimization and users can directly obtain the calibrated by using it.

In [ ]:
optQ0ZAmp, optQ1ZAmp, optCZLen, optCondPhaseLoss = czCaliCondPhase(
    sche=model, q0=q0, q1=q1, maxIter=50)

print(f"The optimal loss value is {optCondPhaseLoss}")
print(f"The optimal amplitude of Z pulse on qubit {q0} is {optQ0ZAmp}")
print(f"The optimal amplitude of Z pulse on qubit {q1} is {optQ1ZAmp}")
print(f"The optimal amplitude of duration of Z pulses is {optCZLen} ns")

Next, we modify the pulse length `czLen` and the Z pulse amplitudes of the first and second qubit `q0ZAmp`, `q1ZAmp` in the configuration of `model` for later use:

In [ ]:
model.conf["caliDataCZ"][(0, 1)]["q0ZAmp"] = optQ0ZAmp
model.conf["caliDataCZ"][(0, 1)]["q1ZAmp"] = optQ1ZAmp
model.conf["caliDataCZ"][(0, 1)]["czLen"] = optCZLen

Here we plot the pulse sequences for calibration:

In [ ]:
condPhaseJobList = czCaliCondPhaseJob(model, q0, q1, optQ0ZAmp, optQ1ZAmp, optCZLen)
print(r"When the second qubit is initialized to |1>:")
condPhaseJobList.jobs[0].plot()
print(r"When the second qubit is initialized to |0>:")
condPhaseJobList.jobs[1].plot()

### 3. Calibrate the Dynamical-Phase

In the above steps, we apply the magnetic flux to generate the conditional phase of $\pi$. But at the same time, the applied magnetic flux will also produce dynamical phase accumulation on each qubit, so in this section, we aim to design the control pulses to compensate for the dynamical phase.

Here, we use the virtual-$Z$ (VZ) gate to achieve the above compensation. The basic principle of the VZ gate is to adjust the phase of the arbitrary wave generator (AWG) to realize the operation similar to the rotation along the $z$-axis. For example, we want to perform two successive $X_{\theta}$ operations, however, the second X operation has a phase $\phi$ shift with respect to the first $X$ operation, i.e.:

$$
X^{(\phi_0)}_{\theta} X_{\theta} = e^{-i\theta(\hat{\sigma}_x\cos\phi_0+\hat{\sigma}_y\sin\phi_0) / 2} X_{\theta} = Z_{-\phi_0}X_{\theta}Z_{\phi_0}X_{\theta}.
$$

Since the measurement of the qubit in the superconducting system is carried out along the $z$-axis, this makes the final $Z_{-\phi_0}$ operation has no effect on the observables. Therefore, it can be seen that the effect of adjusting the AWG phase is equivalent to adding Z pulse between the two X gates.

In this tutorial, we use the following circuit to achieve pulse calibration:

![fig:czCali_dynamical_phase_circuit](figures/cali-cz-dynamics-phase.png)

Where $Z_{\theta_1}$ and $Z_{\theta_2}$ are implemented using VZ gates. According to the above quantum circuit and the values of $\theta_1$ and $\theta_2$, Quanlse Scheduler is used to prepare the required pulse sequence. We use VZ gates to implement $Z_{\theta_1}$ and $Z_{\theta_2}$. Subsequently, we calculate the evolution simulation result, and use the infidelity between the final state and the ideal Bell state $(|00\rangle + |11\rangle) / \sqrt{2}$ as the loss function for optimization. Similarly, Quanlse 2.1 encapsulates the above functions. Users can directly call the `czCaliDynamicalPhase()` function in the `Quanlse.Calibration.TwoQubit` module to calibrate the dynamical phase, which will output the optimal phase shift amount $\theta_1^*$ and $\theta_2^*$, as well as the infidelity of the ideal Bell state:

In [ ]:
optQ0VZPhase, optQ1VZPhase, optDynaPhaseLoss = czCaliDynamicalPhase(
    sche=model, q0=q0, q1=q1, method="Nelder-Mead", q0VZPhaseInit=0., q1VZPhaseInit=0.)

print(f"The optimal loss value is {optDynaPhaseLoss}")
print(f"The optimal phase correction on qubit {q0} is {optQ0VZPhase / 2 / pi} * 2pi")
print(f"The optimal phase correction on qubit {q1} is {optQ1VZPhase / 2 / pi} * 2pi")

It is worth noting that in the above steps, we can use techniques such as Randomized benchmarking or Quantum process tomography to replace the method of calculating Bell distortion in step 3 above to optimize the phase to obtain more accurate results.

Finally, we store the calibrated phase information in `model`:

In [ ]:
model.conf["caliDataCZ"][(0, 1)]["q0VZPhase"] = optQ0VZPhase
model.conf["caliDataCZ"][(0, 1)]["q1VZPhase"] = optQ1VZPhase

## Generating Bell State Using Calibrated Pulses

With the previous steps, the pulses required for the CZ gate have been calibrated. Now, we can use the calibrated waveforms to compile a given quantum circuit. Users can use all the features of **Quanlse Scheduler** directly through the `model` object. We can first use the `model.clearCircuit()` method to clear the defined quantum circuit in the current model. Then add the quantum circuit to prepare the Bell state, and call the `model.schedule()` method to compile and generate the required pulse sequences. Here, the compilation process will call the previously saved pulse parameters to generate the pulses, thereby generating a pulse sequence with higher fidelity:

In [ ]:
# Clear the circuit
model.clearCircuit()

# Define the circuit
H(model.Q[0])
H(model.Q[1])
CZ(model.Q[0], model.Q[1])
H(model.Q[1])

# Generate the ideal unitary of the quantum circuit
uIdeal = model.getMatrix()

# Generate the pulse for the circuit
jobBell = model.schedule()
jobBell.plot()

Then we can use the `model.simulate()` method and pass in the pulse task `jobBell`, the initial state and the number of repetitions to simulate evolution, and get the count of each ground state in the final state:

In [ ]:
# Calculate final state
finalState = model.simulate(
    job=jobBell, state0=basis(model.sysLevel ** model.subSysNum, 0), shot=1000)

# Print the population distance of Bell State
pop = project(numpy.square(abs(finalState[0]["state"])).T[0], model.subSysNum, model.sysLevel, 2)
stateIdeal = uIdeal @ basis(uIdeal.shape[0], 0).T[0]
popIdeal = numpy.square(abs(stateIdeal))
print("Distance of real and ideal Bell states:", numpy.sum(numpy.abs(pop - popIdeal)) / len(pop))

# Plot the population of computational basis
plotBarGraph(computationalBasisList(2, 3), finalState[0]["population"], 
             "Counts of the computational basis", "Computational Basis", "Counts")

Here, we generate a high-fidelity Bell state with the calibrated single-qubit gates and CZ gates. In real quantum computers, we can further improve the fidelity of CZ gates utilizing quantum process tomography or randomized benchmarking technologies.

## Summary

After reading this tutorial on CZ gate pulse calibration, the users could follow this link [tutorial-calibration-cz.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-calibration-cz.ipynb) to the GitHub page of this Jupyter Notebook document and obtain the relevant code to run this program for themselves. The users are encouraged to explore other advanced research which is different from this tutorial.

## References

\[1\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)

\[2\] [Yuan, Xu, et al. "High-Fidelity, High-Scalability Two-Qubit Gate Scheme for Superconducting Qubits." *Physical Review Letters* 125 (2020): 240503 .](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.125.240503)